In [1]:
import tensorflow as tf
import sonnet as snt
import cv2 as cv

print('tf: '+tf.__version__)
print('cv: '+cv.__version__)
print('snt: '+snt.__version__)

tf: 2.6.1
cv: 4.5.4
snt: 2.0.0


In [ ]:
actions = os.listdir('../hmdb51_org')

for video_name in os.listdir('../hmdb51_org/fall_floor'):
   # try:
        cap=cv.VideoCapture('../hmdb51_org/fall_floor/'+video_name)
        while cap.isOpened():
            ret, frame=cap.read()
            if not ret:
                print("Can't receive frame (stream end?). Exiting...")
                break
            cv.imshow('frame',frame)
            if cv.waitKey(1)==ord('q'):
                break

        cap.release()
        cv.destroyAllWindows()
  #  except Exception as e:
   #     print(f'Failed on {video_name}')

We get the first train and test videos.

In [3]:
import re

splits_files= os.listdir('../test_train_splits')
train_split1=[]
test_split1=[]
r = re.compile('1\.txt$')
first_split_files=list(filter(r.search,splits_files))

for file in first_split_files:
    f=open('../test_train_splits/'+file,'r')
    for line in f:
        if(re.search('1 $',line)):
            train_split1.append(line.split(' ',1)[0])
        elif(re.search('2 $',line)):
            test_split1.append(line)

    f.close()
